In [63]:
rm( list=ls() )  #Borro todos los objetos
gc()   #Garbage Collection

library("data.table")
library("rpart")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,704428,37.7,1321401,70.6,1321401,70.6
Vcells,1317855,10.1,83075154,633.9,103058080,786.3


In [64]:
# particionar agrega una columna llamada fold (por defecto) a un dataset
# que consiste en una particion estratificada segun agrupa
# particionar(  data=dataset
              #,division=c(70,30)
              #, agrupa=clase_ternaria
              #, seed=semilla) crea una particion 70, 30 

particionar  <- function(
  data #el dataset sobre el que se trabaja, va a tener una nueva columna
  ,division #este parametro determina las proporciones, vector de N cantidad
  ,agrupa="" #aca va el criterio de agrupacion
  ,campo="fold" #el nombre del nuevo campo, default "fold"
  ,start=1 #los numeros asignados parten de 1 por defecto
  ,seed=NA) #seed default NA
{
  if(!is.na(seed)) set.seed(seed)

  bloque <- unlist(
    mapply(
      function(x, y) {rep(y, x)}
      ,division
      ,seq(from = start
        ,length.out = length(division)
      )
    )
  )
  #esto ayuda a visualizar lo que hace la funcion

  #modelo de como es un bloque
  walA = bloque 
  #el modelo repetido segun la cantidad de rows
  walB = rep( bloque, ceiling(nrow(data)/length(bloque))) 
  #la muestra de ese modelo que ahora lleva otro orden
  walC = sample( walB)
  #preparo el retorno
  listReturn <- list(walA,walB,walC)
  data[ , (campo) :=  sample( rep( bloque, ceiling(.N/length(bloque))))[1:.N], by= agrupa ]
  
  return(listReturn)
}

In [65]:
#apunto el working directory al directorio donde esta ubicado el script
setwd(".\\") 
getwd() #solo para validar

[1] "d:/Onedrive/Austral/Cursos/08 - Laboratorio de Implementacion 1/labo2023v/src/rpart"

In [66]:
#cargo los datos
dataset  <- fread("./datasets/dataset_pequeno.csv")
cat("Lineas en el archivo original: ", nrow(dataset), "\n")

#trabajo solo con los datos con clase, es decir 202107
dataset  <- dataset[ clase_ternaria!= "" ]
cat("Lineas con clase: ", nrow(dataset), "\n")

Lineas en el archivo original:  329919 
Lineas con clase:  164682 


In [67]:
#particiono estratificadamente el dataset
#Cambiar por la primer semilla de cada uno!
a = particionar( data=dataset, division=c(7,3), agrupa="clase_ternaria", seed= 262139 )  #Cambiar por la primer semilla de cada uno !

In [68]:
cat("Modelo Bloque: ", unlist(a[1]), "\n")
cat("Bloque Repetido: ", unlist(unlist(a[2])[1:40]), "...\n")
cat("Bloque Repetido: ", unlist(unlist(a[3])[1:40]), "...\n")

Modelo Bloque:  1 1 1 1 1 1 1 2 2 2 
Bloque Repetido:  1 1 1 1 1 1 1 2 2 2 1 1 1 1 1 1 1 2 2 2 1 1 1 1 1 1 1 2 2 2 1 1 1 1 1 1 1 2 2 2 ...
Bloque Repetido:  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 2 2 1 1 2 1 2 1 1 1 2 1 1 1 1 1 2 1 ...


In [69]:
param_basicos  <- list( "cp"=         -1,  #complejidad minima
                        "minsplit"=  400,  #minima cantidad de registros en un nodo para hacer el split
                        "minbucket"=  10,  #minima cantidad de registros en una hoja
                        "maxdepth"=    8 ) #profundidad máxima del arbol

In [70]:
#genero el modelo
modelo  <- rpart("clase_ternaria ~ .",     #quiero predecir clase_ternaria a partir del resto
                 data= dataset[ fold==1],  #fold==1  es training,  el 70% de los datos
                 xval= 0,
                 control=  param_basicos )  #aqui van los parametros

In [71]:
#aplico el modelo a los datos de testing
prediccion  <- predict( modelo,   #el modelo que genere recien
                        dataset[ fold==2],  #fold==2  es testing, el 30% de los datos
                        type= "prob") #type= "prob"  es que devuelva la probabilidad

#prediccion es una matriz con TRES columnas, llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
#cada columna es el vector de probabilidades 

In [72]:
#agrego una columna que es la de las ganancias
dataset[  , ganancia :=  ifelse( clase_ternaria=="BAJA+2", 117000, -3000 ) ]

In [73]:
#para testing agrego la probabilidad
dataset[ fold==2 , prob_baja2 := prediccion[, "BAJA+2"] ]

In [74]:
#calculo la ganancia en testing  qu es fold==2
ganancia_test  <- dataset[ fold==2 & prob_baja2 >  0.025, sum(ganancia) ]

In [75]:
#escalo la ganancia como si fuera todo el dataset
ganancia_test_normalizada  <-  ganancia_test / 0.3

estimulos  <- dataset[ fold==2 & prob_baja2 > 0.025 , .N ]
aciertos   <- dataset[ fold==2 & prob_baja2 > 0.025 & clase_ternaria =="BAJA+2", .N ]

In [ ]:
cat( "Testing total: ",  dataset[ fold==2, .N ], "\n" )
cat( "Testing BAJA+2: ", dataset[ fold==2 & clase_ternaria =="BAJA+2", .N ], "\n" )

cat( "Estimulos: ", estimulos, "\n" )
cat( "Aciertos (BAJA+2): ",  aciertos,  "\n" )

cat( "Ganancia en testing (normalizada): ", ganancia_test_normalizada, "\n" )